# Demo for Problem 1

### Read csv data

In [177]:
import pandas as pd

borrower_df = pd.read_csv('csv/borrower_data.csv')
loan_df = pd.read_csv('csv/loan_data.csv')
repayment_df = pd.read_csv('csv/repayment_data.csv')
schedule_df = pd.read_csv('csv/schedule_data.csv')

### Sqlite uses dates in DD-MM-YYYY format

In [178]:
def convert_dates(df, col):
    df[col] = pd.to_datetime(df[col], format='%m/%d/%Y')
    df[col] = df[col].dt.strftime('%Y-%m-%d')
    return df

In [179]:
for col in ['Date_of_release', 'Maturity_date']:
    loan_df = convert_dates(loan_df, col)
repayment_df = convert_dates(repayment_df, 'Date_paid')
schedule_df = convert_dates(schedule_df, 'Expected_payment_date')

### Inspect the Dataframes

In [180]:
borrower_df

,Borrower_Id,State,City,zipcode,borrower_credit_score
0,123fd35,dfgc,12olki,19473,4
1,123fd36,rtgio,89hfg,923475,a


In [181]:
loan_df

,Borrower_id,loan_id,Date_of_release,Term,InterestRate,LoanAmount,Downpayment,Payment_frequency,Maturity_date
0,123fd35,32u09wekjbfje,2021-01-27,12,1.05,1209484,124993,1.007903e+05,2022-01-31
1,123fd36,9190i0-nbfb,2021-02-01,24,1.05,12394031,1000000,5.164180e+05,2023-02-28
2,123fd35,09u924rbwf,2021-04-16,36,1.05,45784937,1200000,1.271804e+06,2024-04-30
3,123fd36,3240-9rfwb,2021-03-26,12,1.05,1234890,132000,1.029075e+05,2022-03-31


In [182]:
repayment_df.head()

,loan_id,payment_id,Date_paid,Amount_paid
0,32u09wekjbfje,3434r409kmPAID123456,2021-03-31,100790.3333
1,32u09wekjbfje,3434r409kmPAID123457,2021-03-31,100790.3333
2,32u09wekjbfje,3434r409kmPAID123458,2021-04-27,100790.3333
3,32u09wekjbfje,3434r409kmPAID123459,2021-05-27,100790.3333
4,32u09wekjbfje,3434r409kmPAID123460,2021-06-27,100790.3333


In [183]:
schedule_df.head()

,loan_id,schedule_id,Expected_payment_date,Expected_payment_amount
0,32u09wekjbfje,3434r409km123456,2021-02-27,100790.3333
1,32u09wekjbfje,3434r409km123457,2021-03-27,100790.3333
2,32u09wekjbfje,3434r409km123458,2021-04-27,100790.3333
3,32u09wekjbfje,3434r409km123459,2021-05-27,100790.3333
4,32u09wekjbfje,3434r409km123460,2021-06-27,100790.3333


### Insert DataFrames into Sqlite as tables

In [184]:
import sqlite3

conn = sqlite3.connect('loan_data.db')

borrower_df.to_sql('borrower', conn, if_exists='replace', index=False)
loan_df.to_sql('loan', conn, if_exists='replace', index=False)
repayment_df.to_sql('repayment', conn, if_exists='replace', index=False)
schedule_df.to_sql('schedule', conn, if_exists='replace', index=False)

84

### Query to calculate PAR Days for each loan payment

In [187]:
_q = """
WITH repayment_cte AS (
    SELECT 
        R.loan_id,
        REPLACE(R.payment_id, 'PAID', '') AS payment_id,
        R.Date_paid, 
        S.Expected_payment_date,
        JULIANDAY(R.Date_paid) - JULIANDAY(S.Expected_payment_date) AS par_days
    FROM repayment R
    JOIN schedule S 
    ON R.loan_id = S.loan_id 
    AND REPLACE(R.payment_id, 'PAID', '') = S.schedule_id
)

SELECT * FROM repayment_cte
"""

rep = pd.read_sql_query(_q, conn)
rep.head(10)

,loan_id,payment_id,Date_paid,Expected_payment_date,par_days
0,32u09wekjbfje,3434r409km123456,2021-03-31,2021-02-27,32.0
1,32u09wekjbfje,3434r409km123457,2021-03-31,2021-03-27,4.0
2,32u09wekjbfje,3434r409km123458,2021-04-27,2021-04-27,0.0
3,32u09wekjbfje,3434r409km123459,2021-05-27,2021-05-27,0.0
4,32u09wekjbfje,3434r409km123460,2021-06-27,2021-06-27,0.0
5,32u09wekjbfje,3434r409km123461,2021-07-31,2021-07-27,4.0
6,32u09wekjbfje,3434r409km123462,2021-10-31,2021-08-27,65.0
7,32u09wekjbfje,3434r409km123463,2021-10-31,2021-09-27,34.0
8,32u09wekjbfje,3434r409km123464,2021-10-27,2021-10-27,0.0
9,32u09wekjbfje,3434r409km123465,2021-11-27,2021-11-27,0.0


### Final Query as Defined in Problem 1: (Data Engineering Technical Interview.docx)

In [188]:
final_query = """
SELECT 
    L.loan_id,
    L.Borrower_id,
    L.Date_of_release AS loan_date_of_release,
    L.Term,
    L.LoanAmount,
    L.Downpayment,
    B.State,
    B.City,
    B.zipcode,
    B.borrower_credit_score,
    L.Payment_frequency,
    L.Maturity_date,
    JULIANDAY(MAX(RP.Date_paid)) - JULIANDAY(MAX(S.Expected_payment_date)) AS current_days_past_due,
    MAX(S.Expected_payment_date) AS last_due_date,
    MAX(RP.Date_paid) AS last_repayment_date,
    (SUM(S.Expected_payment_amount) - SUM(RP.Amount_paid)) AS amount_at_risk,
    SUM(RP.Amount_paid) AS total_amount_paid,
    SUM(S.Expected_payment_amount) AS total_amount_expected
FROM 
    borrower B
JOIN 
    loan L ON B.Borrower_id = L.Borrower_id
JOIN 
    schedule S ON L.loan_id = S.loan_id
JOIN 
    repayment RP ON L.loan_id = RP.loan_id
GROUP BY 
    L.loan_id, L.Borrower_id
"""

df = pd.read_sql_query(final_query, conn)
df

,loan_id,Borrower_id,loan_date_of_release,Term,LoanAmount,Downpayment,State,City,zipcode,borrower_credit_score,Payment_frequency,Maturity_date,current_days_past_due,last_due_date,last_repayment_date,amount_at_risk,total_amount_paid,total_amount_expected
0,09u924rbwf,123fd35,2021-04-16,36,45784937,1200000,dfgc,12olki,19473,4,1.271804e+06,2024-04-30,0.0,2024-04-16,2024-04-16,0.0,1.648258e+09,1.648258e+09
1,3240-9rfwb,123fd36,2021-03-26,12,1234890,132000,rtgio,89hfg,923475,a,1.029075e+05,2022-03-31,0.0,2022-03-26,2022-03-26,0.0,1.481868e+07,1.481868e+07
2,32u09wekjbfje,123fd35,2021-01-27,12,1209484,124993,dfgc,12olki,19473,4,1.007903e+05,2022-01-31,0.0,2022-01-27,2022-01-27,0.0,1.451381e+07,1.451381e+07
3,9190i0-nbfb,123fd36,2021-02-01,24,12394031,1000000,rtgio,89hfg,923475,a,5.164180e+05,2023-02-28,0.0,2023-02-02,2023-02-02,0.0,2.974567e+08,2.974567e+08
